<a href="https://colab.research.google.com/github/aemrhb/IPI_project-seminar/blob/main/yoloV8_deepsort_wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd gdrive/My Drive/ps-ss23

/content/gdrive/My Drive/ps-ss23


In [ ]:
!git clone https://github.com/computervisioneng/deep_sort.git

fatal: destination path 'deep_sort' already exists and is not an empty directory.


In [ ]:
import os
import random
import cv2

! pip install ultralytics
from ultralytics import YOLO

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.5/585.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 24.4 MB/s eta 0:00:00


In [ ]:
from deep_sort.deep_sort.tracker import Tracker 
from deep_sort.deep_sort import nn_matching
from deep_sort.tools import generate_detections as gdet
from deep_sort.deep_sort.detection import Detection

In [ ]:
# Definition of the parameters
max_cosine_distance = 0.5
nn_budget = None
nms_max_overlap = 1.0

#initialize deep sort
model_filename = 'model_data/mars-small128.pb'
encoder = gdet.create_box_encoder(model_filename, batch_size=1)
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
tracker = Tracker(metric)

In [ ]:
model = YOLO("yolov8n.pt")

In [ ]:
video_path = os.path.join('.', 'data', 'people.mp4')
video_out_path = os.path.join('.', 'out.mp4')

In [ ]:
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()

In [ ]:
cap_out = cv2.VideoWriter(video_out_path, cv2.VideoWriter_fourcc(*'MP4V'), cap.get(cv2.CAP_PROP_FPS),
                          (frame.shape[1], frame.shape[0]))

AttributeError: ignored

In [ ]:
import numpy as np

In [ ]:
# colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for j in range(10)]
ret, frame = cap.read()
detection_threshold = 0.5
while ret:

    results = model(frame)

    for result in results:
        detections = []
        for r in result.boxes.data.tolist():
          x1, y1, x2, y2, score, class_id = r
          # print(r)
          x1 = int(x1)
          x2 = int(x2)
          y1 = int(y1)
          y2 = int(y2)
          class_id = int(class_id)
          if score > detection_threshold:
              detections.append([x1, y1, x2, y2, score])

        bboxes = np.asarray([d[:-1] for d in detections])
        # print(bboxes.shape)
        bboxes[:, 2:] = bboxes[:, 2:] - bboxes[:, 0:2]
        # print(bboxes.shape)
        scores = [d[-1] for d in detections]
        features = encoder(frame, bboxes)
        dets = []
        for bbox_id, bbox in enumerate(bboxes):
          # print(bbox,'s',scores[bbox_id],'f',features[bbox_id])
          dets.append(Detection(bbox, scores[bbox_id], features[bbox_id]))
          # dets.append([bbox,scores[bbox_id],features[bbox_id]])
        print((dets[0:2]))
        print(type(dets),"class")
        
        # print(type(dets))
 
        tracker.predict()
        tracker.update(dets)

        result = []
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            bbox = track.to_tlbr()
            id = track.track_id

            result.append([ id, bbox[0], bbox[1], bbox[2], bbox[3]])
        print(result)


      



        # for track in tracker.tracks:
        #     bbox = track.bbox
        #     x1, y1, x2, y2 = bbox
        #     track_id = track.track_id

        #     cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (colors[track_id % len(colors)]), 3)

    cap_out.write(frame)
    ret, frame = cap.read()

cap.release()
cap_out.release()
cv2.destroyAllWindows()


0: 384x640 34 persons, 3 birds, 167.1ms
Speed: 7.5ms preprocess, 167.1ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)



[<deep_sort.deep_sort.detection.Detection object at 0x7f003548aec0>, <deep_sort.deep_sort.detection.Detection object at 0x7f01681645b0>]
<class 'list'> class
[[1, 774.745783237894, 99.33784399910525, 813.1876458959503, 182.74619282627964], [16, 621.2257470490289, 444.19530720034084, 674.3474289011314, 570.5562309326368], [34, 592.3556034103308, 53.90533497321037, 625.7029957208724, 135.62721313184346], [36, 924.2619156480716, 4.184273313742715, 952.8676839272682, 98.76456237154392], [42, 701.7067738978182, 31.244225677872613, 724.5998403335216, 109.14203891478402], [50, 852.4574133101303, -0.5191084962784771, 884.56116890275, 79.5501221680589], [54, 497.8267201523639, 61.277915874328286, 532.9818268022713, 155.08129656292266], [58, 76.87915082347686, 172.25723316701828, 129.51378685816778, 266.39489561890844], [60, 620.9175663460251, 116.55351196109969, 660.4697738047682, 207.46382980134058], [61, 1027.256982103082, 11.833225962786202, 1071.0514809853553, 96.06722380330143], [70, 507.1

0: 384x640 35 persons, 3 birds, 191.6ms
Speed: 4.4ms preprocess, 191.6ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)



[<deep_sort.deep_sort.detection.Detection object at 0x7f00354f4c10>, <deep_sort.deep_sort.detection.Detection object at 0x7f00354f4310>]
<class 'list'> class
[[1, 773.2638071152488, 98.91341623639869, 810.847009693011, 185.1084310740822], [34, 592.5695583845835, 55.710149798881005, 625.5326409136021, 137.93010577253062], [60, 617.4792614841631, 119.47651070134808, 656.4913593698303, 209.61674256608023], [75, 500.73415115366356, 293.6011219299929, 546.5320801514135, 398.26291896720215], [86, 1112.576125028343, 6.806615437100582, 1158.2592929399248, 90.03553755197487], [89, 1074.8710241384667, 10.011322230215477, 1102.3366613857477, 83.56877683394518], [94, 941.6982695048036, 474.5677116398874, 1007.5301579229698, 596.8652800564385], [102, 1193.8054629630278, 98.58028617506778, 1235.4261911810113, 182.88013355732687], [105, 748.9394392477259, 622.4672917499169, 836.4107687138475, 720.0016471459113]]


0: 384x640 36 persons, 2 birds, 217.0ms
Speed: 6.3ms preprocess, 217.0ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)



...

[<deep_sort.deep_sort.detection.Detection object at 0x7f003548aec0>, <deep_sort.deep_sort.detection.Detection object at 0x7f003548ace0>]
<class 'list'> class
[[36, 923.0967606650604, 2.1743528268287022, 952.3098613654247, 99.22885728524963], [105, 671.0627444619607, 40.201637160550405, 698.5255270706215, 116.11594418130827], [120, 619.4647229403587, 435.1372305270027, 672.2171466132305, 560.7129607109358], [135, 701.612460464273, 34.52197413149581, 724.2601959398229, 112.11967061793247], [142, 851.8923656742513, 2.9456140185327726, 883.9128762636026, 82.25483304002776], [145, 496.79223077649743, 63.364829449674204, 532.8256454478599, 158.91871671842014], [150, 79.41302167996116, 169.693847996854, 132.59133886477716, 265.5379168437686], [152, 629.4090607001935, 108.10126073675723, 670.6859179113749, 202.09598735450027], [153, 1031.7558355371405, 13.231490908540444, 1075.6566513323698, 96.85705059853825], [162, 506.72830776083987, 256.0323199226178, 550.9903462337967, 357.37259965152066]

0: 384x640 31 persons, 1 bird, 224.0ms
Speed: 5.6ms preprocess, 224.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)



[<deep_sort.deep_sort.detection.Detection object at 0x7f01681662c0>, <deep_sort.deep_sort.detection.Detection object at 0x7f003548bca0>]
<class 'list'> class
[[36, 923.4963780313215, 3.4409105920062544, 952.4011038213526, 99.02185986563376], [120, 621.7032858019086, 438.60625922435577, 672.3048373582136, 558.0581652881133], [135, 701.7625189752101, 34.445570680137294, 724.2630171940681, 111.70733586470921], [142, 852.3890878666215, 1.8901821241245926, 884.398773408055, 81.48089932855802], [145, 497.290365938007, 63.30682253360797, 532.7764635046259, 157.78187815080028], [150, 78.64243249438775, 170.73417557005726, 131.76204091298536, 266.47059911389533], [152, 630.1372912572318, 109.7917482378596, 670.9135218745396, 203.04493087939272], [153, 1029.944655302127, 12.796851962013847, 1073.804316082625, 96.72291346167215], [162, 506.63477560136454, 254.5819508311225, 551.4046187605209, 356.86828229994956], [169, 10.665280257335269, 85.89447469973733, 48.31654962682037, 167.72757079245264],

0: 384x640 32 persons, 1 bird, 184.2ms
Speed: 2.8ms preprocess, 184.2ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)


[<deep_sort.deep_sort.detection.Detection object at 0x7f003548bdc0>, <deep_sort.deep_sort.detection.Detection object at 0x7f003548b850>]
<class 'list'> class
[[36, 923.9835762624095, 3.9046199836709548, 952.7527047906453, 98.95142806521565], [120, 620.9618869006811, 441.13077762959307, 673.9155520341034, 566.9304538474576], [135, 701.648159022363, 32.490922236480365, 724.4273088832425, 110.2742856252585], [142, 852.0976546752156, 0.8789922049673251, 884.3680388366844, 81.25666233951142], [145, 497.50112076813787, 62.67596954670576, 532.7110523625989, 156.7528087547987], [150, 77.69291434702673, 171.16734536560637, 130.48093102040394, 265.51927226553414], [152, 631.0980223852853, 110.37179672263206, 671.6000594718081, 203.3589994306708], [153, 1029.4851939666878, 12.663238048768918, 1073.0304383487992, 96.69739466853606], [162, 506.84671755114215, 254.12089057158553, 551.9599214369105, 356.77852572314725], [169, 10.230259861130545, 85.98651298845479, 47.78695079225945, 167.9719935805307

In [ ]:
# colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for j in range(10)]
ret, frame = cap.read()
detection_threshold = 0.5
while ret:

    results = model(frame)

    for result in results:
        detections = []
        for r in result.boxes.data.tolist():
          x1, y1, x2, y2, score, class_id = r
          # print(r)
          x1 = int(x1)
          x2 = int(x2)
          y1 = int(y1)
          y2 = int(y2)
          class_id = int(class_id)
          if score > detection_threshold:
              detections.append([x1, y1, x2, y2, score])

        bboxes = np.asarray([d[:-1] for d in detections])
        # print(bboxes.shape)
        bboxes[:, 2:] = bboxes[:, 2:] - bboxes[:, 0:2]
        # print(bboxes.shape)
        scores = [d[-1] for d in detections]
        features = encoder(frame, bboxes)
        dets = []
        for bbox_id, bbox in enumerate(bboxes):
          # print(bbox,'s',scores[bbox_id],'f',features[bbox_id])
          dets.append(Detection(bbox, scores[bbox_id], features[bbox_id]))
          # dets.append([bbox,scores[bbox_id],features[bbox_id]])
        print((dets[0:2]))
        print(type(dets),"class")
        
        # print(type(dets))
 
        tracker.predict()
        tracker.update(dets)

        result = []
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            bbox = track.to_tlbr()
            id = track.track_id

            result.append([ id, bbox[0], bbox[1], bbox[2], bbox[3]])
        print(result)


      



        # for track in tracker.tracks:
        #     bbox = track.bbox
        #     x1, y1, x2, y2 = bbox
        #     track_id = track.track_id

        #     cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (colors[track_id % len(colors)]), 3)

    cap_out.write(frame)
    ret, frame = cap.read()

cap.release()
cap_out.release()
cv2.destroyAllWindows()